In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_cliente"

nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

### DDL - Tabela de Clientes
Cria tabela Silver de cadastro de clientes:
- **cd_cliente** (PK - STRING): Identificador unico do cliente
- **nm_cliente**: Nome completo
- **ds_email**: Email de contato
- **nm_regiao**: Regiao geografica
- **nu_idade**: Idade (usada para criar faixa etaria na camada Gold)

**Importante**: cd_cliente e STRING (nao BIGINT) para flexibilidade com diferentes formatos de ID.

### Configuracao
Define catalogo, schema e nome da tabela Silver de clientes.

In [0]:
query = f"""
CREATE OR REPLACE TABLE {nome_tabela_destino} (
    cd_cliente STRING COMMENT 'Identificador único do cliente.',
    nm_cliente VARCHAR(150) COMMENT 'Nome completo do cliente.',
    ds_email VARCHAR(150) COMMENT 'Endereço de e-mail.',
    nm_regiao VARCHAR(50) COMMENT 'Região geográfica do cliente.',
    nu_idade SMALLINT COMMENT 'Idade do cliente.',
    dt_ingestion TIMESTAMP,
    dc_origem STRING,
    CONSTRAINT pk_cliente PRIMARY KEY (cd_cliente)
)
COMMENT 'Tabela silver de cadastro de clientes';
"""

spark.sql(query)
print(f"✅ Tabela {nome_tabela_destino} criada com sucesso!")